In [35]:
# import pandas as pd
# import numpy as np
# # CSV 파일 불러오기
# senti_df = pd.read_csv("/Users/Shared/최종선_교수님/Face_skin_disease/감성분석/감성사전/senti_labeled_df.csv")
# ntoken_df = pd.read_csv("/Users/Shared/최종선_교수님/Face_skin_disease/데이터 전처리/피부 질환 화장품 데이터/여드름_스킨케어/크림/Ntoken_review.csv")


In [36]:
# # senti_labeled_df.csv 의 pred 열만 추출
# pred_col = senti_df[["pred"]]

# # 두 데이터프레임 합치기 (인덱스 기준)
# merged_df = pd.concat([ntoken_df, pred_col], axis=1)

# # 결과 저장
# merged_df.to_csv("merged_output.csv", index=False)

# print("병합 완료! merged_output.csv 생성됨.")

In [37]:
## ===================================================================
## 1. 기본 설정 및 라이브러리 임포트
## ===================================================================
import numpy as np
import pandas as pd
from ast import literal_eval
import torch
from typing import Optional, List, Dict

# 추천 모델용
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# 감성 분석 모델용
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Surprise (협업 필터링용)
try:
    from surprise import Dataset, Reader, SVD
    HAS_SURPRISE = True
except ImportError:
    HAS_SURPRISE = False
    print("⚠️ Surprise가 설치되지 않아 협업 필터링(SVD)을 건너뜁니다. 필요시 %pip install scikit-surprise 실행하세요.")

print("✅ 모든 라이브러리 임포트 완료.")

✅ 모든 라이브러리 임포트 완료.


In [38]:
# ===================================================================
# 2. 데이터 준비 및 특성 생성
# ===================================================================
file_path = "merged_output.csv"
df = pd.read_csv(file_path)

# 사용자 확인 컬럼: product_name, customer_name, skin_type, skin_tone, 
#                  skin_concern_1, skin_concern_2, review, review_date(date), rating, pred
# gender, tokens, Ntoken_review 등도 포함 확인

df.rename(columns={
    'customer_name': 'user_id',
    'product_name': 'item_id'
}, inplace=True)

# 'tokens' 컬럼의 리스트를 공백으로 연결하는 함수
def join_tokens(x):
    try:
        x_list = literal_eval(x) if isinstance(x, str) else x
        if isinstance(x_list, list):
            return " ".join(map(str, x_list))
    except (ValueError, SyntaxError):
        pass
    return str(x) if pd.notna(x) else ""

# 콘텐츠 기반 추천을 위한 텍스트 통합
df["text"] = df["review"].fillna("") + " " + (df["Ntoken_review"].apply(join_tokens) if "Ntoken_review" in df.columns else "")

# 협업 필터링을 위한 평점 보정
df["rating_aug"] = (df["rating"].astype(float) + 0.5 * df.get("pred", 0)).clip(0.5, 5.0)

# 아이템별 정보 집계 및 인덱스 생성
item_text_df = df.groupby("item_id")["text"].apply(lambda s: " ".join(map(str, s))).reset_index()
item_to_idx = {item_id: i for i, item_id in enumerate(item_text_df["item_id"])}
idx_to_item = {i: item_id for item_id, i in item_to_idx.items()}

print(f"✅ 데이터 준비 완료 (총 {len(df)}개 리뷰, {len(item_text_df)}개 제품)")
display(df.head())

✅ 데이터 준비 완료 (총 11795개 리뷰, 20개 제품)


,item_id,user_id,skin_type,skin_tone,skin_concerns,review,date,rating,gender,tokens,Ntoken_review,pred,text,rating_aug
0,라로슈포제 시카플라스트 멀티 리페어 크림 100ml (+15ml+시카밤3ml+시카선...,vwaaang,건성,쿨톤,각질 / 모공,제품을 만난 이뢰로 계속 재구매중입니다 흡수가 빠르고 얼굴에 자고 일어나면 얼굴에 ...,2025.09.17,5.0,여성,"[('제품', 'NNG'), ('을', 'JKO'), ('만나', 'VV'), ('...","['제품', '구매', '흡수', '얼굴', '얼굴', '유분기', '여드름', '...",0,제품을 만난 이뢰로 계속 재구매중입니다 흡수가 빠르고 얼굴에 자고 일어나면 얼굴에 ...,5.0
1,라로슈포제 시카플라스트 멀티 리페어 크림 100ml (+15ml+시카밤3ml+시카선...,나이거가사로써도돼,건성,웜톤,민감성 / 트러블,아마도 수부지인 피부인데 이거 바르고 피부 다 뒤집어져서 한번 쓰고 바로 다른 잘맞...,2025.09.07,1.0,여성,"[('아마도', 'MAG'), ('수부지', 'NNG'), ('이', 'VCP'),...","['수부지', '피부', '피부', '사람']",0,아마도 수부지인 피부인데 이거 바르고 피부 다 뒤집어져서 한번 쓰고 바로 다른 잘맞...,1.0
2,라로슈포제 시카플라스트 멀티 리페어 크림 100ml (+15ml+시카밤3ml+시카선...,찍지말라옹,복합성,겨울쿨톤,각질 / 모공,와 정말 순하네요 가격이 쎄서 살까말까하다가 결국 샀는데 후회 도 없어요 저는 ...,2025.09.25,5.0,여성,"[('오', 'VV'), ('아', 'EC'), ('정말', 'MAG'), ('순하...","['가격', '후회']",0,와 정말 순하네요 가격이 쎄서 살까말까하다가 결국 샀는데 후회 도 없어요 저는 ...,5.0
3,라로슈포제 시카플라스트 멀티 리페어 크림 100ml (+15ml+시카밤3ml+시카선...,꿀피부촉촉,복합성,NaN,NaN,재재재구매 피부장벽 튼튼해집니다 트러블 관련 세럼에 이거 드음뿍 아낌없이 바릅니다...,2025.09.26,5.0,여성,"[('재', 'XPN'), ('재', 'XPN'), ('재', 'XPN'), ('구...","['구매', '피부장벽', '트러블', '관련', '트러블', '완화', '장벽',...",0,재재재구매 피부장벽 튼튼해집니다 트러블 관련 세럼에 이거 드음뿍 아낌없이 바릅니다...,5.0
4,라로슈포제 시카플라스트 멀티 리페어 크림 100ml (+15ml+시카밤3ml+시카선...,악악악악지성,지성,봄웜톤,각질 / 다크서클,극지성에 심각한 여드름 피부라 시어버터같은 여드름 유발하는 성분이나 모공막는 성분 ...,2025.10.01,3.0,여성,"[('극', 'NNG'), ('지성', 'NNG'), ('에', 'JKB'), ('...","['지성', '여드름', '피부', '시어버터', '여드름', '유발', '성분',...",0,극지성에 심각한 여드름 피부라 시어버터같은 여드름 유발하는 성분이나 모공막는 성분 ...,3.0


In [39]:
# ===================================================================
# 3. 콘텐츠 기반 추천 (KoBERT)
# ===================================================================
MODEL_NAME = "monologg/kobert"
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu"))
print(f"KoBERT 모델을 '{DEVICE}'에서 실행합니다.")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()

@torch.no_grad()
def encode_texts(texts: list, batch_size: int = 16) -> np.ndarray:
    output_vectors = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encodings = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors="pt").to(DEVICE)
        mean_pooled_emb = bert_model(**encodings).last_hidden_state.mean(dim=1)
        output_vectors.append(torch.nn.functional.normalize(mean_pooled_emb, p=2, dim=1).cpu())
    return torch.vstack(output_vectors).numpy()

X_item_txt = encode_texts(item_text_df["text"].tolist())

def get_user_text_profile(df: pd.DataFrame, user_id: str) -> Optional[np.ndarray]:
    liked_reviews = df[(df.user_id == user_id) & (df.rating >= 4.0)]
    item_indices = [item_to_idx[i] for i in liked_reviews.item_id if i in item_to_idx]
    if not item_indices: return None
    weights = liked_reviews.loc[liked_reviews.item_id.isin(item_to_idx), "rating"].values
    weights = (weights / (weights.sum() + 1e-8))[:, None]
    profile_vector = (X_item_txt[item_indices] * weights).sum(0, keepdims=True)
    return profile_vector / (np.linalg.norm(profile_vector, axis=1, keepdims=True) + 1e-9)

def recommend_content_based(df: pd.DataFrame, user_id: str, k: int = 1000) -> Dict[str, float]:
    user_profile = get_user_text_profile(df, user_id)
    if user_profile is None: return {}
    similarities = cosine_similarity(user_profile, X_item_txt).ravel()
    return {idx_to_item[i]: float(similarities[i]) for i in np.argsort(similarities)[::-1][:k]}

print(f"✅ KoBERT 벡터 생성 및 콘텐츠 기반 추천 함수 정의 완료")

KoBERT 모델을 'mps'에서 실행합니다.
✅ KoBERT 벡터 생성 및 콘텐츠 기반 추천 함수 정의 완료


In [40]:
# ===================================================================
# 4. 협업 필터링 추천 (SVD)
# ===================================================================
svd_model = None
if HAS_SURPRISE:
    reader = Reader(rating_scale=(0.5, 5.0))
    data = Dataset.load_from_df(df[["user_id", "item_id", "rating_aug"]], reader)
    trainset = data.build_full_trainset()
    svd_model = SVD(n_factors=64, n_epochs=20, random_state=42)
    svd_model.fit(trainset)
    print("✅ SVD 모델 학습 완료.")

def recommend_collaborative_filtering(user_id: str, all_items: list) -> Dict[str, float]:
    if svd_model is None: return {}
    predictions = [(item, float(svd_model.predict(user_id, item).est)) for item in all_items]
    return dict(predictions)

print("✅ 협업 필터링 추천 함수 정의 완료.")

✅ SVD 모델 학습 완료.
✅ 협업 필터링 추천 함수 정의 완료.


In [41]:
# ===================================================================
# 5. 피부 적합도 분석 및 필터링 함수 (수정)
# ===================================================================
# (수정) skin_concerns 컬럼을 사용하도록 변경
for col in ["skin_type", "skin_tone", "skin_concerns"]:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown").astype(str)

def get_ratio_pivot(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    if column_name not in df.columns:
        return pd.DataFrame(index=item_text_df["item_id"])
    counts = df.groupby(["item_id", column_name])["user_id"].count().reset_index(name="count")
    totals = counts.groupby("item_id")["count"].transform("sum").clip(lower=1)
    counts["ratio"] = counts["count"] / totals
    pivot = counts.pivot(index="item_id", columns=column_name, values="ratio").fillna(0.0)
    return pivot.reindex(item_text_df["item_id"]).fillna(0.0)

R_skin_type = get_ratio_pivot(df, "skin_type")
R_skin_tone = get_ratio_pivot(df, "skin_tone")
# (수정) skin_concerns 컬럼으로 피봇 테이블 생성
R_skin_concerns = get_ratio_pivot(df, "skin_concerns") 

print("✅ 제품별 피부 프로필(타입, 톤, 고민) 집계 완료.")

# (수정) 필터링 함수가 skin_concerns 컬럼을 사용하도록 변경
def filter_items_by_skin_compatibility(user_id: str, threshold: float = 0.1) -> set:
    user_info = df[df.user_id == user_id].tail(1)
    if user_info.empty:
        return set(item_text_df["item_id"])

    user_skin_type = str(user_info.skin_type.values[0])
    user_skin_tone = str(user_info.skin_tone.values[0])
    user_concern = str(user_info.skin_concerns.values[0]) # skin_concerns 사용
    
    candidate_items = set()
    
    if user_skin_type in R_skin_type.columns:
        candidate_items.update(R_skin_type[R_skin_type[user_skin_type] >= threshold].index)
    if user_skin_tone in R_skin_tone.columns:
        candidate_items.update(R_skin_tone[R_skin_tone[user_skin_tone] >= threshold].index)
    # (수정) R_skin_concerns 피봇 테이블을 사용하여 필터링
    if user_concern in R_skin_concerns.columns:
        candidate_items.update(R_skin_concerns[R_skin_concerns[user_concern] >= threshold].index)
        
    return candidate_items if candidate_items else set(item_text_df["item_id"])

print("✅ 피부 적합도(타입, 톤, 고민) 기반 필터링 함수 정의 완료.")

✅ 제품별 피부 프로필(타입, 톤, 고민) 집계 완료.
✅ 피부 적합도(타입, 톤, 고민) 기반 필터링 함수 정의 완료.


In [42]:
# ===================================================================
# 6. 최종 하이브리드 추천
# ===================================================================
def normalize_scores(score_dict: Dict) -> Dict:
    if not score_dict: return {}
    values = np.array(list(score_dict.values()), dtype=float)
    min_val, max_val = values.min(), values.max()
    if max_val - min_val < 1e-8: return {k: 0.5 for k in score_dict}
    return {k: float((v - min_val) / (max_val - min_val)) for k, v in score_dict.items()}

def recommend_hybrid_skin_filtered(user_id: str, k: int = 10, content_weight: float = 0.5, skin_filter_threshold: float = 0.1):
    candidate_items = filter_items_by_skin_compatibility(user_id, threshold=skin_filter_threshold)
    print(f"👤 사용자 '{user_id}'의 피부 정보와 맞는 후보 제품 {len(candidate_items)}개를 필터링했습니다.")
    if not candidate_items:
        print("⚠️ 피부에 맞는 제품을 찾지 못해 전체 제품을 대상으로 추천합니다.")
        candidate_items = set(item_text_df["item_id"])

    seen_items = set(df.loc[df.user_id == user_id, "item_id"])
    candidate_items = list(candidate_items - seen_items)

    content_scores_all = recommend_content_based(df, user_id)
    cf_scores_all = recommend_collaborative_filtering(user_id, item_text_df["item_id"].tolist())
    
    content_scores = {item: score for item, score in content_scores_all.items() if item in candidate_items}
    cf_scores = {item: score for item, score in cf_scores_all.items() if item in candidate_items}
    
    content_scores_norm = normalize_scores(content_scores)
    cf_scores_norm = normalize_scores(cf_scores)
    
    final_scores = []
    for item in candidate_items:
        score = (
            content_weight * content_scores_norm.get(item, 0) +
            (1 - content_weight) * cf_scores_norm.get(item, 0)
        )
        final_scores.append((item, score))
        
    final_scores.sort(key=lambda x: x[1], reverse=True)
    
    return pd.DataFrame(final_scores[:k], columns=["item_id", "final_score"])

print("✅ 피부 필터링 기반 하이브리드 추천 함수 정의 완료.")

✅ 피부 필터링 기반 하이브리드 추천 함수 정의 완료.


In [43]:
# ===================================================================
# 7. 추천 시스템 실행 예시
# ===================================================================
if not df.empty:
    example_user_id = df["user_id"].iloc[0]

    print(f"\n--- [최종 결과] 피부 필터링 기반 하이브리드 추천 ---")
    print(f"피부 필터링 기준(threshold): 특정 피부타입/톤/고민을 가진 리뷰어 비율이 10% 이상인 제품만 후보로 선정")

    recommendations = recommend_hybrid_skin_filtered(
        user_id=example_user_id, 
        k=10, 
        content_weight=0.6, 
        skin_filter_threshold=0.1
    )

    display(recommendations)
else:
    print("데이터프레임이 비어 있어 추천을 실행할 수 없습니다.")


--- [최종 결과] 피부 필터링 기반 하이브리드 추천 ---
피부 필터링 기준(threshold): 특정 피부타입/톤/고민을 가진 리뷰어 비율이 10% 이상인 제품만 후보로 선정
👤 사용자 'vwaaang'의 피부 정보와 맞는 후보 제품 12개를 필터링했습니다.


,item_id,final_score
0,마몽드 블루 캐모마일 크림 60ml 기획 (+30ml),0.879244
1,[아랑 PICK] 낫츠 센텔라스카 연고 일랑일랑 15g,0.832843
2,[손상/장벽 리페어] 아벤느 시칼파트 플러스 SOS 리페어 크림 100ml 2입 기획,0.752454
3,이즈앤트리 어니언 뉴페어 겔크림 80ml 대용량 기획 (+겔크림20ml+패드2매),0.726204
4,[응급진정/38% 여드름 개선] 로벡틴 시카케어 블레미쉬 클리어링 크림 50ml 리...,0.654759
5,[강희재픽/추가증정한정기획] 아벤느 시칼파트 플러스 S.O.S 크림 100ml 2입...,0.621607
6,키엘 울트라 훼이셜 크림 28ml,0.599900
7,[대용량기획] 한율 쑥시카 수분크림 110ml (+25ml+ 패드2매+흡착팩폼2ml),0.578260
8,키엘 울트라 훼이셜 오일-프리 젤 크림 50ml,0.400000
9,[김고은 Pick] 가히 엑스틴C밤,0.362142
